Prepare Data for Searching

In [1]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext() 
sc.stop()
config = sc.getConf()
config.set('spark.cores.max','4')
config.set('spark.executor.memory', '8G')
config.set('spark.driver.maxResultSize', '8g')
config.set('spark.kryoserializer.buffer.max', '512m')
config.set("spark.driver.cores", "4")
sc.stop()
sc = SparkContext(conf = config) 
sqlContext = SQLContext(sc)
print("Using Apache Spark Version", sc.version)

23/04/25 16:12:02 WARN Utils: Your hostname, Johannes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.206.53.80 instead (on interface en0)
23/04/25 16:12:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 16:12:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Apache Spark Version 3.3.2


/Users/johannebear/opt/anaconda3/envs/Flask/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# Read the csv dataset into Spark Dataframe and output the rows in the dataframe
all_videos_file='/Users/johannebear/Library/Mobile Documents/com~apple~CloudDocs/APPLIED ANALYTICS/APAN 5400/final project/youtube videos datasets/YouTubeVideos_trending_data.csv'
all_videos_sdf = sqlContext.read.option("header", "true") \
                         .option("delimiter", ",") \
                         .option("inferSchema", "true") \
                         .csv(all_videos_file)
all_videos_sdf.count()

2985417

In [3]:
# Filter out all the videos in USA
usa_videos=all_videos_sdf.filter(all_videos_sdf['country']=='US')

In [4]:
# Show the first 10 rows of videos in usa alongwith title, view_couunt,likes, and category
usa_videos.select('video_id', 'view_count', 'likes', 'category').show(10)

+-----------+----------+------+--------------+
|   video_id|view_count| likes|      category|
+-----------+----------+------+--------------+
|3C66w5Z0ixs|   1514614|156908|People & Blogs|
|M9Pmf9AB4Mo|   2381688|146739|        Gaming|
|J78aPJ3VyNs|   2038853|353787| Entertainment|
|kXLn3HkpjaA|    496771| 23251|         Music|
|VIUo6yapDbc|   1123889| 45802| Howto & Style|
|w-aidBdvZo8|    949491| 77487| Entertainment|
|uet14uf9NsE|    470446| 47990| Howto & Style|
|ua4QMFQATco|   1050143| 89190|     Education|
|SnsPZj91R7E|   1402687| 95694| Entertainment|
|SsWHMAhshPQ|    741028|113983|         Music|
+-----------+----------+------+--------------+
only showing top 10 rows



In [5]:
from pyspark.sql import SparkSession
# Create a sparksession object
spark = SparkSession.builder.appName("Convert to Pandas").getOrCreate()
# Convert the Spark DataFrame to a Pandas DataFrame
usa_videos_pandas_df = usa_videos.toPandas()
# Print the rows of Pandas DataFrame
print(len(usa_videos_pandas_df))


23/04/25 16:12:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


192590


In [6]:
import pandas as pd
# Write the usa_videos dataframe to a csv file
usa_videos_pandas_df.to_csv('/Users/johannebear/Library/Mobile Documents/com~apple~CloudDocs/APPLIED ANALYTICS/APAN 5400/final project/youtube videos datasets/USA_Videos_trending_data.csv')
print('write successfully!')


write successfully!


Import Filtered Dataset into Neo4j Database for Query

In [7]:
from neo4j import GraphDatabase

# Connect to database
database_name = "neo4j"
username = "apan5400"
password = "12345678"
uri = "bolt://localhost:7689/" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


In [8]:
print(usa_videos_pandas_df.columns)


Index(['video_id', 'video_title', 'publishedAt', 'country', 'channelId',
       'channelTitle', 'category', 'trending_date', 'tags', 'view_count',
       'likes', 'dislikes', 'comment_count', 'thumbnail_link',
       'comments_disabled', 'ratings_disabled', 'description'],
      dtype='object')


In [9]:
# Delete all nodes and relationships
query1 = ("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r")
result1 = session.run(query1)
print("All Nodes and relationships are deleted!")

All Nodes and relationships are deleted!


In [10]:
# Import graph from csv and create relationships
query2 = (
    "LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-f46884dd-b89d-4a4e-8dca-2a9c5950888d/USA_Videos_trending_data.csv' AS line "
    'CREATE (videos:Videos { \
                       id: line.video_id, \
                title: line.video_title,\
                  tags: line.tags,\
                       description: line.description\
                            } \
            )'
    'CREATE(performance:Performance{\
                          view_count: toInteger(line.view_count), \
                          likes: toInteger(line.likes),\
                         dislikes: toInteger(line.dislikes),\
                       comment_count: toInteger(line.comment_count)\
                                   } \
            )'
     'MERGE  (category:Category {name: line.category})'  
     'CREATE (videos)-[:BELONG_TO]->(category)'
     'CREATE (videos)-[:HAS_PERFORMANCE]->(performance)'
     'CREATE (performance)-[:BELONG_TO]->(category)'
    )

result2 = session.run(query2)
print("All videos are imported from a csv file!")

All videos are imported from a csv file!


How many videos in each category？

In [11]:
query3=('MATCH (videos:Videos)-[:BELONG_TO]->(category:Category) \
RETURN category.name, count(videos) AS num_videos \
ORDER BY num_videos DESC')
result3 = session.run(query3)   
[f"{record['category.name']}: {record['num_videos']}" for record in result3]


['Entertainment: 38511',
 'Gaming: 37795',
 'Music: 31490',
 'Sports: 21419',
 'People & Blogs: 16830',
 'Comedy: 10146',
 'Film & Animation: 7612',
 'News & Politics: 7296',
 'Science & Technology: 5833',
 'Howto & Style: 5269',
 'Education: 4698',
 'Autos & Vehicles: 3638',
 'Travel & Events: 1073',
 'Pets & Animals: 866',
 'Nonprofits & Activism: 114']

How many total count_views in each category?

In [12]:
query4=('MATCH (videos:Videos)-[:HAS_PERFORMANCE]->(performance:Performance)-[:BELONG_TO]->(category:Category) \
        RETURN category.name, SUM(performance.view_count) AS total_views \
           ORDER BY total_views DESC')

result4 = session.run(query4)   
[f"{record['category.name']}: {record['total_views']}" for record in result4]

['Music: 137989415350',
 'Entertainment: 116419483155',
 'Gaming: 65480643859',
 'Sports: 42990992839',
 'People & Blogs: 29540072442',
 'Film & Animation: 17890817016',
 'Comedy: 17537271734',
 'News & Politics: 16058843040',
 'Science & Technology: 15520265077',
 'Education: 8190372273',
 'Howto & Style: 6655100772',
 'Autos & Vehicles: 3460024257',
 'Travel & Events: 1094905011',
 'Pets & Animals: 1008237756',
 'Nonprofits & Activism: 279268359']

Print the top 10 videos with most comments

In [13]:
query5=('MATCH (videos:Videos)-[:HAS_PERFORMANCE]->(performance:Performance) \
RETURN videos.title, performance.comments \
ORDER BY performance.comments DESC \
LIMIT 10')
result5 = session.run(query5)   
[f"{record['videos.title']}: {record['performance.comments']}" for record in result5]

['I ASKED HER TO BE MY GIRLFRIEND...: None',
 'Apex Legends | Stories from the Outlands – “The Endorsement”: None',
 'I left youtube for a month and THIS is what happened.: None',
 'XXL 2020 Freshman Class Revealed - Official Announcement: None',
 'Ultimate DIY Home Movie Theater for The LaBrant Family!: None',
 "I Haven't Been Honest About My Injury.. Here's THE TRUTH: None",
 'OUR FIRST FAMILY INTRO!!: None',
 'CGP Grey was WRONG: None',
 "SURPRISING MY DAD WITH HIS DREAM TRUCK!! | Louie's Life: None",
 'Ovi x Natanael Cano x Aleman x Big Soto - Vengo De Nada [Official Video]: None']

What are the top 10 most frequent words appeared in the top 1000 most viewed videos'tags in the sports category

In [14]:
query6 = (
    "MATCH (videos:Videos)-[:BELONG_TO]->(category:Category{name: 'Pets & Animals'}) "
    "WHERE size(videos.tags) > 0 "
    "WITH videos "
    "ORDER BY videos.views DESC "
    "LIMIT 100 "
    "UNWIND split(videos.tags, '|') as tag "
    "WITH TOLOWER(tag) as lowercase_tag "
    "WITH lowercase_tag, count(*) as tag_count "
    "WHERE lowercase_tag <> '[none]' AND lowercase_tag <> ''"
    "RETURN lowercase_tag, tag_count "
    "ORDER BY tag_count DESC "
    "LIMIT 10"
)
result6 = session.run(query6)
[{record["lowercase_tag"]:record['tag_count']} for record in result6]

[{'tucker budzyn': 22},
 {'my dog reacts': 21},
 {'dog reviews food': 21},
 {'tucker taste test': 21},
 {'golden retriever': 19},
 {'jealous dog reaction': 18},
 {'funny dog reaction': 17},
 {'animals': 16},
 {'tucker the dog': 16},
 {'dog reviews food with girlfriend': 15}]

Build the Flask web search engine for top 10 highest frequency tags of top 1000 videos in each category

In [15]:
#  Create a flask application instance
from flask import Flask, render_template, request
app = Flask(__name__)

driver = GraphDatabase.driver("bolt://localhost:7689", auth=("apan5400", "12345678"))



In [16]:
mkdir templates

mkdir: templates: File exists


In [17]:
# Build a function that input a category then return 10 most frequent tags
def get_top_tags(category_name):
    with driver.session() as session:
        query = (
            f"MATCH (videos:Videos)-[:BELONG_TO]->(category:Category{{name: '{category_name}'}}) "
            "WHERE size(videos.tags) > 0 "
            "WITH videos "
            "ORDER BY videos.views DESC "
            "LIMIT 1000 "
            "UNWIND split(videos.tags, '|') as tag "
            "WITH TOLOWER(tag) as lowercase_tag "
            "WITH lowercase_tag, count(*) as tag_count "
            "WHERE lowercase_tag <> '[none]' AND lowercase_tag <> ''"
            "RETURN lowercase_tag, tag_count "
            "ORDER BY tag_count DESC "
            "LIMIT 10"
        )
        result = session.run(query)
        tags = [{record["lowercase_tag"]: record["tag_count"]} for record in result]
        return tags


In [18]:
get_top_tags('Sports')

[{'football': 184},
 {'highlights': 162},
 {'nba': 161},
 {'sports': 131},
 {'soccer': 124},
 {'basketball': 114},
 {'boxing': 86},
 {'espn': 76},
 {'nfl': 70},
 {'ufc': 61}]

In [19]:
# Create a flask route

app.view_functions.pop('home', None)
app.view_functions.pop('search', None)
app.view_functions.pop('result', None)

@app.route('/', methods=['GET', 'POST'])
def search():
    return render_template('search.html')

In [20]:
@app.route('/results', methods=['GET', 'POST'])
def result():
    print(request.method) # Print out the request method
    if request.method == 'POST':
        category_name = request.form['category']
        result = get_top_tags(category_name)
        return render_template('result.html', result=result, category=category_name)
    else:
        return render_template('search.html'), 200

In [ ]:
if __name__ == '__main__':
    app.config['DEBUG'] = True
    app.run()